# Домашнее  задание  № 1.  
*Графематический и морфологический анализ текста*

# **Фейгина Анастасия**

**Вариант D**

**Задание:**

Вычислить и проанализировать статистику морфологической омонимии в РЯ, взяв для этого 2-3 текста разных стилей/жанров
и составив программу, которая на базе выбранного, программно подключаемого морфопроцессора определяет:

– общее количество словоупотреблений, число различных словоформ, количество уникальных лемм, число незнакомых слов,
коэффициент лексического богатства текста (= отношение числа различных лемм к общему числу словоупотреблений);

–  абсолютную и относительную (с учетом/без учета неизменяемых слов) частоту омонимичных словоформ, абсолютную и
относительную частоту словоформ с лексико-морфологической омонимией, максимальное и среднее число омонимов у 
словоформ текста, словоформы с наибольшим числом омонимов, наиболее частотный омоним

В качестве подключаемого морфопроцессора можно взять, например:
  mystem(http://api.yandex.ru/mystem/downloads/) или АОТ (http://www.aot.ru )


**Решение:**

Для проведения анализа были выбраны 2 текста разных жанров (научный и художественный). Тексты приведены в файлах kuprin.txt, neuron.txt

В качестве подключаемого морфопроцессора я выбрала Mystem и pymorphy2 (https://pymorphy2.readthedocs.org/en/latest/#). 

В первом случае выбор обоснован интересом к продукции Яндекса, второй - более широким функционалом. Поэтому часть заданий реализовано с помощью Mystem, а другая часть с помощью Pymorph.

**Подготовка данных:**

Прежде чем начинать анализировать данные, нужно их "почистить", привести в удобный для работы вид. Изначально мы имеем тесты со знаками препинания, цифрами, пробелами и другими символами. Нас интересуют лишь слова, поэтому была написана функция *make_clear_text*. Рассмотрим ее работу подробнее:

In [1]:
# подключение необходимых библиотек
import pymorphy2
from texts import *
from pymystem3 import Mystem

In [2]:
def make_clear_text(text):
    # текст без спец символов/цифр/пунктуации
    m = Mystem()
    clear_text = []
    # переводим все заглавные буквы в строчные
    lm = m.analyze(text.lower())
    for i in range(0, len(lm)):
        # в этом месте мы отбрасываем все посторонные символы, остается массив слов
        if 'analysis' in lm[i]:
            clear_text.append(lm[i]['text'])
    return clear_text

def no_one_symbol_text(text):
    # убираем слова состоящие из одного символа
    new_text = list(text)
    for i in text:
        if len(i) <= 1:
            new_text.remove(i)
    return new_text

Для написания этой функции был использован морфологический процессор Mystem. Он анализирует все элементы текста, расставляя тег "analysis" только словам, игнорируя все остальные символы. Этим я и воспользовалась. Хотя изначально была идея использовать регулярные выражения. 
Теперь все готово для проведения анализа, начнем с **общего количества совоупотреблений**.

In [3]:
def usage_count(text):
    # общее количество словоупотреблений
    # приводим данные в удобный вид, считаем, сколько слов в нашем тексте
    result = len(make_clear_text(text))
    print("Общее количество словоупотреблений: ", result)
    return result

Далее реализована функция, которая считает **количество различных словоформ**. Для этого необходимо сделать множество словоупотреблений, таким образом мы уберем повторяющиеся элементы.

In [4]:
def count_different_word_forms(text):
    # число различных словоформ
    # приводим данные в удобный вид, получаем список всех слов в тексте
    clear_text = make_clear_text(text)
    # находим множество всех слов в тексте (теперь нет повторяющихся слов) и посчитаем их количество
    result = len(set(clear_text))
    print("Число различных словоформ: ", result)

Теперь реализуем функцию подсчитывающую **количество уникальных лемм**. Здесь я использовала морфопроцессор Mystem чтобы найти леммы. 

In [5]:
def number_of_unique_lemmas(text):
    # количество уникальных лемм
    m = Mystem()
    # текст с леммами
    lemmas = m.lemmatize(text)
    # делим текст на слова
    lemmatized_text = ''.join(lemmas)
    # чистим данные от лишних символов
    clear_text = no_one_symbol_text(make_clear_text(lemmatized_text))
    result = len(set(clear_text))
    print("Количество уникальных лемм: ", result)
    return result

Следующим пунктом найдем **количество неизвестных слов**. У морфопроцессора **pymorph** есть очень удобная функция **word_is_known(word)**, она возвращает **True**, если слово известно, иначе - **False**. 

In [6]:
def number_of_unknown_words(text):
    # число незнакомых слов
    morph = pymorphy2.MorphAnalyzer()
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    # суммируем количество неизвестных слов
    result = sum([not morph.word_is_known(x) for x in clear_text])
    print("Число незнакомых слов: ", result)

Далее рассмотрим метод вычисления коэффициента **лексического богатства текста**. Здесь очень просто, потому что мы уже умеем считать количество уникальных лемм и общее количество словоупотреблений. Осталось только разделить их друг на друга.

In [7]:
def lexical_richness_of_the_text_index(text):
    # коэффициент лексического богатства текста
    result = number_of_unique_lemmas(text)/usage_count(text)
    print("Коэффициент лексического богатства текста:", result)

Рассмотрим функцию **абсолютной и относительной частоты омонимичных словоформ** без учета неизменяемых слов. Здесь нам нужно посчитать все слова у которых парсер выдает более одного типа разбора + надо учесть и отбросить те омонимы, у которых лексемы одинаковые. Для этого мы берем очередной омоним и склоняем все его разборы, чтобы найти лексемы. Далее находим множества этих лексем, и если оказывается, что в таких множествах 1 элемент - такой омоним неизменяемый, мы его отбрасываем. Но такой способ не всегда срабатывает. Существуют сокращениях слов (например "руб" - рубль), "руб" является неизменяемым омонимом, но когда мы начнем его склонять, морфопроцессор посчитает, что это "рубль", и склонять мы будем уже совсем не то, что хотели. Поэтому будем использовать встроенный в морфопроцессор тег m.tag если в нем есть параметр Fixd, то это неизменяемое слово. Опытным путем подтвердилось, что такой тег гарантирует работу для сокращений.

In [8]:
def frequency_homonymous_word_forms(text):
    # абсолютная и относительная (без учета/с учетом неизменяемых слов) частота омонимичных словоформ
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    for j in clear_text:
        m = morph.parse(j)
        for k in range(0, len(m)):
            lexeme = []
            unchangeable_tag = []
            unchangeable_tag.append('Fixd' in m[k].tag)
            unchangeable = []
            lex = m[k].lexeme
            for i in lex:
                lexeme.append(i.word)
            lexeme = set(lexeme)
            unchangeable.append(len(lexeme) == 1)
        unchangeable = unchangeable + unchangeable_tag
        if len(m) > 1 and True not in unchangeable and morph.word_is_known(j):
            result += 1
    print("Абсолютная частота омонимичных словоформ без учета неизменяемых слов: ", result)
    print("Относительная частота омонимичных словоформ без учета неизменяемых слов: ", result/len(make_clear_text(text)))

Функция **абсолютной и относительной частоты омонимичных словоформ** c учетом неизменяемых слов. С учетом неизменяемых слов все эти действия просто не выполняем, нас интересуют как неизменяемые так и изменяемые слова.

In [9]:
def frequency_homonymous_word_forms_full(text):
    # абсолютная и относительная (с учетом неизменяемых слов) частота омонимичных словоформ
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    for j in clear_text:
        m = morph.parse(j)
        if len(m) > 1 and morph.word_is_known(j):
            result += 1
    print("Абсолютная частота омонимичных словоформ с учетом неизменяемых слов: ", result)
    print("Относительная частота омонимичных словоформ с учетом неизменяемых слов: ", result/len(make_clear_text(text)))

Лексико-морфологическая омонимя - совпадение словоформ двух разных лексем. Применив метод parse к очередному слову, получаем список разборов (для омонимов), в каждом таком разборе имеется различная информация, в том числе и некий индивидуальный номер. Если у слов совпадают эти номера, то и лексемы у них одинаковые. В данном методе необходимо проверить номера всех разборов слова, и если они не совпадают, то такой омоним нам подходит. 

In [10]:
def frequency_of_word_forms_with_lexicalmorphological_homonymy(text):
    # абсолютная и относительнаю частота словоформ с лексико-морфологической омонимией
    morph = pymorphy2.MorphAnalyzer()
    result = 0
    clear_text = set(no_one_symbol_text(make_clear_text(text)))
    set_of_lexem = []
    for j in clear_text:
        m = morph.parse(j)
        if morph.word_is_known(j):
            set_of_lexem = set([m[i][4][0][2] for i in range(0, len(m))])
        if len(m) > 1 and len(set_of_lexem) > 1:
            result += 1
    print("Абсолютная частота словоформ с лексико-морфологической омонимией: ", result)
    print("Относительная частота словоформ с лексико-морфологической омонимией: ", result/len(make_clear_text(text)))

Теперь рассмотрим функцию, которая находит **максимальное и среднее число омонимов у словоформ текста, словоформу с наибольшим числом омонимов, наиболее частотный омоним**.

Для того чтобы найти максимальное число омонимов нужно запоминать текущее число омонимов у очередного слова и сравнимать с предыдущим максимальным числом, если текущее больше чем предыдущее, сделаем его максимальным и так далее. Перебрав таким образом все слова, надем максимум.

Чтобы найти среднее нужно просто суммировать количество омонимов каждого слова потом разделить на их количество.

Словоформа с наибольшим числом омонимов находится как и максимальное число омонимов: мы просто каждый раз запоминаем слово вместе с числом.

С наиболее частотным омонимом дела обстоят сложнее, тут нужно хранить словарик, у которого ключи это слова, а значения ключей - число, означающее сколько раз нам такое слово встретилось. Если слово нам еще не встречалось, мы добавляем в словарь новый ключ, если слово оказалось омонимом, то значение этого ключа увеличивается на единицу. В конце обработки текста находим наибольшее значение и показываем как результат его ключ те самый популярый омоним. 

In [11]:
def maximum_and_average_number_of_homonyms_in_the_text_word_forms(text):
    # максимальное и среднее число омонимов у словоформ текста
    morph = pymorphy2.MorphAnalyzer()
    result = {}
    max_homonym = 0
    mean_homonym = 0
    counter = 0
    clear_text = no_one_symbol_text(make_clear_text(text))
    for j in clear_text:
        if j not in result:
            result[j] = 0
        m = morph.parse(j)
        length = len([y for y in m if 'Fixd' not in y.tag])
        if length > 1 and morph.word_is_known(j):
            result[j] += 1
            counter += 1
            mean_homonym += length
            if length > max_homonym:
                max_homonym = length
                wordform_with_max_homonym = j
    print("Максимальное число омонимов у словоформ: ", max_homonym)
    print("Среднее число омонимов у словоформ: ", mean_homonym/counter)
    print("Словоформа с наибольшим числом омонимов: ", wordform_with_max_homonym)
    print("Наиболее частотный омоним: ",  max(result, key=result.get))

**Анализ текстов:**

Теперь применим все написанные функции к текстам

Научный текст ("Физиологии человека" под  редакцией В. М. Покровского, Г. Ф. Коротько)

In [12]:
count_different_word_forms(scientific_text)
lexical_richness_of_the_text_index(scientific_text)
number_of_unknown_words(scientific_text)
frequency_homonymous_word_forms_full(scientific_text)
frequency_homonymous_word_forms(scientific_text)
frequency_of_word_forms_with_lexicalmorphological_homonymy(scientific_text)
maximum_and_average_number_of_homonyms_in_the_text_word_forms(scientific_text)

Число различных словоформ:  4409
Количество уникальных лемм:  2277
Общее количество словоупотреблений:  14290
Коэффициент лексического богатства текста: 0.15934219734079777
Число незнакомых слов:  412
Абсолютная частота омонимичных словоформ с учетом неизменяемых слов:  2289
Относительная частота омонимичных словоформ с учетом неизменяемых слов:  0.1601819454163751
Абсолютная частота омонимичных словоформ без учета неизменяемых слов:  2221
Относительная частота омонимичных словоформ без учета неизменяемых слов:  0.15542337298810358
Абсолютная частота словоформ с лексико-морфологической омонимией:  592
Относительная частота словоформ с лексико-морфологической омонимией:  0.04142757172848146
Максимальное число омонимов у словоформ:  13
Среднее число омонимов у словоформ:  3.0057971014492755
Словоформа с наибольшим числом омонимов:  любой
Наиболее частотный омоним:  на


Художественный текст ("Гранатовый браслет" А. И. Куприн)

In [13]:
count_different_word_forms(artistic_text)
lexical_richness_of_the_text_index(artistic_text)
number_of_unknown_words(artistic_text)
frequency_homonymous_word_forms_full(artistic_text)
frequency_homonymous_word_forms(artistic_text)
frequency_of_word_forms_with_lexicalmorphological_homonymy(artistic_text)
maximum_and_average_number_of_homonyms_in_the_text_word_forms(artistic_text)

Число различных словоформ:  5555
Количество уникальных лемм:  3565
Общее количество словоупотреблений:  14069
Коэффициент лексического богатства текста: 0.2533939867794442
Число незнакомых слов:  99
Абсолютная частота омонимичных словоформ с учетом неизменяемых слов:  2717
Относительная частота омонимичных словоформ с учетом неизменяемых слов:  0.1931196247068022
Абсолютная частота омонимичных словоформ без учета неизменяемых слов:  2597
Относительная частота омонимичных словоформ без учета неизменяемых слов:  0.18459023384746606
Абсолютная частота словоформ с лексико-морфологической омонимией:  974
Относительная частота словоформ с лексико-морфологической омонимией:  0.06923022247494491
Максимальное число омонимов у словоформ:  13
Среднее число омонимов у словоформ:  2.8864577173270254
Словоформа с наибольшим числом омонимов:  любой
Наиболее частотный омоним:  на


Тексты примерно выбраны одинакового объема, но после проведения анализа старо очевидно, что художественный текст содержит более чем на 1000 больше различных словоформ, по сравнению с научным текстом. Соответственно и уникальных лемм у него больше. 

Коэффициент лингвистического богатства художественного текста больше в 1.6 раз! 

Однако научный текст обладает большим количеством незнакомых слов, что и следовало ожидать. Разница огромна, научный текст имеет в ~4 раза больше незнакомых слов по сравнению с художественным текстом.

**С учетом неизменяемых слов**

**Научный текст**

        Абсолютная частота омонимичных словоформ:  2289

        Относительная частота омонимичных словоформ:  0.16

**Художественный текст**

        Абсолютная частота омонимичных словоформ:  2717

        Относительная частота омонимичных словоформ:  0.19

Здесь не наблюдается колоссальной разницы, но все же относительная частота показывает, что художественный текст более
богат омонимичными словоформами.

**Без учета неизменяемых слов**

**Научный текст**

        Абсолютная частота омонимичных словоформ:  2221
        Относительная частота омонимичных словоформ:  0.16

**Художественный текст**

        Абсолютная частота омонимичных словоформ:  2597
        Относительная частота омонимичных словоформ:  0.18

Тут разница между значениями стала меньше. Относительная частота художественного текста уменьшилась. Делаем вывод, что неизменяемые слова вносят более существенный вклад в художественный текст, и служат неким признаком художественного жанра.

**Научный текст**


        Абсолютная частота словоформ с лексико-морфологической омонимией:  592
        Относительная частота словоформ с лексико-морфологической омонимией:  0.04
        
**Художественный текст**
   
        Абсолютная частота словоформ с лексико-морфологической омонимией:  974
        Относительная частота словоформ с лексико-морфологической омонимией:  0.07

В данном случае подтверждается гипотеза о том, что художественный текст более богат словоформами с омонимией. 

**Научный текст**

        Максимальное число омонимов у словоформ:  13
        Среднее число омонимов у словоформ:  3.0057971014492755
        Словоформа с наибольшим числом омонимов:  любой
        Наиболее частотный омоним:  на
    
  **Художественный текст**

  
        Максимальное число омонимов у словоформ:  13
        Среднее число омонимов у словоформ:  2.8864577173270254
        Словоформа с наибольшим числом омонимов:  любой
        Наиболее частотный омоним:  на
    

Удивительно, но характер самих омонимов в данном эксперименте почти не отличается!

В процессе выполнения задания я использовала тестовые кусочки различных текстов, они были на много меньше по объему чем те, на которых проведены данные эксерименты. На таких небольших данных характер самих омонимимичных словоформ (максимальное число омонимов у словоформ, среднее число омонимов и т.д.) сильно отличается. Поэтому возможно имеет смысл проводить эксперименты как на всем тексте, так и на отдельных выборках. 

Эксперимент показал, что используя данный подход к анализу текстов, можно получить данные для классификации по жанрам. В дальнейшем определять тип жанра нового текста используя машинное обучение. 